In [1]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML, display, Markdown
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.interpolate import interp1d
from traitlets import directional_link

<h1 align="center">Choose Your Own Universe</h1> 
<h4 align="center">-an N-body interactive simulation of celestial objects-</h4> 


Computer simulations are extremely useful tools that help cosmologists to learn about the formation of structures and galaxies. While there are many codes available that can simulate a vast array of scenarios, this code steps things back to simply study a smaller system of gravitating particles in a box. Fear not, there is still much we can learn from this simulation, particularly about how the particles interact gravitationally with one another and any correlations in their separations as the system is evolved over time.

### How this applet works
This simulation will model the motion of particles, all with the same mass between 0 and 150 $M_{\odot}$, as chosen by you. This mass range encompasses most stars in our universe - from white dwarfs to supergiants. These stars will be contained within a box of length 40 kpc, which is roughly the radius of the Milky Way galaxy.

You will need to choose a configuration for how the stars are positioned to start the simulation, as well as how many stars are in your system and their maximum starting velocities. 

Then, the system will 


In [2]:
layout = widgets.Layout(width = "400px")

In [149]:
# Set up the interactive widgets to assign parameter values 
# A text box to enter the seed for the random no generator
setseed = widgets.IntText(
    value=4080,
    disabled=False
)
display(widgets.HBox([widgets.Label('Set the seed for the random no generator:'), setseed]))

# A dropdown to choose the initial universe configuration 
icwidget = widgets.Dropdown(
    options=['Random Distribution', 'Two Elliptical Galaxies','Lattice (Preset 49 particles)'],
    value='Random Distribution',
    disabled=False,
    layout = layout
)
display(widgets.HBox([widgets.Label('Choose the initial universe configuration:'), icwidget]))

# Slider (integers) to set the number of particles to simulate 
Np_slider = widgets.IntSlider(value = 50, min = 2, max = 100, 
                                 continuous_update=True, layout = layout)
display(widgets.HBox([widgets.Label("No of Particles"), Np_slider]))

def transform(case): # make the lattice case preset 49 particles 
    return {'Lattice (Preset 49 particles)': 49, 'Random Distribution': 30, "Two Elliptical Galaxies": 30}[case]

directional_link((icwidget, 'value'), (Np_slider, 'value'), transform)

# Slider (integers) to set the mass of all the particles in solar masses
pmass_slider = widgets.IntSlider(value = 50, min = 0, max = 150, 
                                 continuous_update=True, layout = layout)
display(widgets.HBox([widgets.Label("Particle mass ($M_\odot$)"), pmass_slider]))

# Slider (floats) to set the maximum drift velocity in units of kpc / Myr 
vmax_slider = widgets.FloatSlider(value = 0.2, min = 0, max = 1,
                                 continuous_update=True, layout = layout)
display(widgets.HBox([widgets.Label("Max drift velocity (kpc/Myr)"), vmax_slider]))

# Toggle to choose 2d or 3d projection for main panel 
projectionmode = widgets.ToggleButtons(
    options=['2D', '3D'],
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Generates a 2D video simulation', 'Generates a 3D video simulation'],
    layout = layout
)
display(widgets.HBox([widgets.Label('Project my universe in:'), projectionmode]))

SEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdDaG9vc2UgdGhlIGluaXRpYWwgdW5pdmVyc2UgY29uZmlndXJhdGlvbjonKSwgRHJvcGRvd24obGF5b3V0PUxheW91dCh3aWR0aD11JzQwMHDigKY=


SEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdQYXJ0aWNsZSBtYXNzICgkTV9cXG9kb3QkKScpLCBJbnRTbGlkZXIodmFsdWU9NTAsIGxheW91dD1MYXlvdXQod2lkdGg9dSc0MDBweCcpLCDigKY=


SEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdNYXggZHJpZnQgdmVsb2NpdHkgKGtwYy9NeXIpJyksIEZsb2F0U2xpZGVyKHZhbHVlPTAuMiwgbGF5b3V0PUxheW91dCh3aWR0aD11JzQwMHDigKY=


SEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdQcm9qZWN0IG15IHVuaXZlcnNlIGluOicpLCBUb2dnbGVCdXR0b25zKGJ1dHRvbl9zdHlsZT11J2luZm8nLCBsYXlvdXQ9TGF5b3V0KHdpZHTigKY=


In [211]:
# Generate a button to run all the code below this cell when clicked after choosing parameters 
from IPython.display import Javascript
Javascript('IPython.notebook.execute_cells_below()')

from IPython.display import Javascript, display
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Simulate my universe!")
button.on_click(run_all)
display(button)

<IPython.core.display.Javascript object>

Button(description=u'Simulate my universe!', style=ButtonStyle())

In [212]:
# Read in the parameters from user input in the widgets 
# The seed for the random number generator
# For reproducibility, set a seed for randomly generated inputs. Change to your favourite integer.
np.random.seed(setseed.value)

# Choice of initial distribution for the universe
initconfig = icwidget.value

# The number of particles to simulate 
if (initconfig == "Lattice (Preset 49 particles)"):
    Np = 49
else:
    Np = Np_slider.value

# The mass of the particles in solar masses
pmass = pmass_slider.value

# The maximum drift velocity 
v_max = vmax_slider.value

# Calculations will be performed for the x, y and z coordinates
Nd = 3

# Projection dimension for the main panel - 2D or 3D 
project = projectionmode.value
if (projectionmode.value == "3D"):
    project_3d = True # project in 3D
else:
    project_3d = False # project in 2D



In [213]:
# Set other parameters and constants that will not be chosen by the user 
# Set gravitational constant 
gconst = 4.3e-6 # kpc M_{sun} (km/s)^2

# Set box length in kpc 
lbox = 20

# Set softening length in kpc
epsilon = 0.01*lbox

#Set the number of particles for the random distribtion to compare against for the correlation function
Nprandom = 200

# Set the total number of timesteps and duration of a timestep 
Nt = 200
dt = 2

# Set how long the animation should dispay each timestep (in milliseconds).
frame_duration = 100

In [214]:
# Set initial positions at random within box depending on the dropdown option chosen 
if (initconfig == "Random Distribution"):
    position = lbox-2*lbox*np.random.random((Nd,Np))
elif (initconfig == "Lattice (Preset 49 particles)"):
    index = 3*np.arange(-3, 4, 1)
    colindex = (np.array([index,]*7))
    rowindex = (np.array([index,]*7).transpose())
    colindex=  colindex.flatten("F")
    rowindex= rowindex.flatten("F")
    position = np.array((rowindex,colindex, np.zeros(Np)))
elif (initconfig == "Two Elliptical Galaxies"):
    cluster1 = np.random.random((Nd,int(np.floor(Np/2))))*3-1.5+5 # Make one random cluster in the top right corner
    cluster2 = np.random.random((Nd,Np-int(np.floor(Np/2))))*3-1.5-5 # Make another in the bottom left corner
    position = np.concatenate((cluster1, cluster2), axis=1) # combine the positions together in a single vector

# Set initial velocities to be random fractions of the maximum
velocity = v_max*(1-2*np.random.random((Nd,Np)))

In [215]:
#Generate a random catalogue to compare against 
randompos = lbox-2*lbox*np.random.random((Nd,Nprandom))

In [216]:
# calculate the separations between each particle
def separation(p): # Function to find separations from position vectors
    s = p[:,None,:] - p[:,:,None] # find N x N x Nd matrix of particle separations
    return np.sum(s**2,axis=0)**0.5 # return N x N matrix of scalar separations

In [217]:
# Create a function to apply boundary conditions
def apply_boundary(p):
    # If the particle leaves the box, make it re-enter but at the opposite side with the same velocity
    p[p>lbox] = -lbox + abs(p[p>lbox]) % lbox # if particles are above/to the right of the box 
    p[p<-lbox] = lbox - abs(p[p<-lbox]) % lbox # if particles are below/to the left of the box 
    return p

In [218]:
# Add newtonian gravity into the simulation
def acceleration(p): # a function to calculate the accelerations of each particle from the position vectors 
    force = np.zeros((Nd,Np))
    # print(np.linalg.norm(diffpos, axis=0))
    for i in np.arange(Np):
        diffpos = p[:,:] - p[:,None,i] # calculate the separations between particle i and all others
        diffpos = diffpos[:,~np.all(diffpos==0, axis=0)] # remove particle i's separations 
        epsilonarray = np.ones(Np-1)*epsilon # Create an array of softening lengths 
        diffposwithepsilon = np.r_[diffpos,[epsilonarray]] # Append softening lengths to separations array
        # Calculate the force on particle i by summing over contributions from all particles in each direction 
        force[:,i] = gconst*pmass**np.sum(diffpos[:,:]/np.linalg.norm(diffposwithepsilon, axis=0)[None,:],axis=1)
        # Only multiplied by one mass since a = F/M so the function can directly return the force vector 
    return force[:,:]

# acceleration(position)
# diffpos = position[:,:] - position[:,None,0] # calculate the separations between particle i and all others
# diffpos = diffpos[:,~np.all(diffpos==0, axis=0)] # remove particle i's separations 
# epsilonarray = np.ones(Np-1)*epsilon # Create an array of softening lengths 
# diffposwithepsilon = np.r_[diffpos,[epsilonarray]] # Append softening lengths to separations array

In [219]:
%%capture 
# ^^ to suppress showing the empty axes 
# Set up the axes on which the points will be shown for panelled plots 

# For the main panel showing the simulation 
plt.ion() # Set interactive mode on
fig = plt.figure(figsize=(14,7)) # Create frame and set size
plt.subplots_adjust(left=0.08, bottom=0.08, right=0.92, top=0.92,wspace=0.15,hspace=0.2)
# Create one set of axes as the left hand panel in a 1x2 grid
if project_3d:
    ax1 = plt.subplot(121,projection='3d') # For very basic 3D projection
    ax1.set_zlim3d(-lbox, lbox)                    # viewrange for z-axis should be [-4,4] 
    ax1.set_ylim3d(-lbox, lbox)                    # viewrange for y-axis should be [-2,2] 
    ax1.set_xlim3d(-lbox, lbox) 
    ax1.set_zlabel("z (kpc)")
else:
    ax1 = plt.subplot(121) # For normal 2D projection
    ax1.set_xlim(-lbox,lbox)  # Set x-axis limits
    ax1.set_ylim(-lbox,lbox)  # Set y-axis limits

ax1.set_ylabel("y (kpc)") # Set axis labels 
ax1.set_xlabel("x (kpc)")

# Create command which will plot the positions of the particles
if project_3d:
    points, = ax1.plot([],[],[],'*',markersize=8)  ## For 3D projection
else:
    points, = ax1.plot([],[],'*',markersize=8) ## For 2D projection

# For the correlation function subplot 
ax2 = plt.subplot(222) # Create second set of axes as the top right panel in a 2x2 grid
xmax = lbox*1.5 # Set xaxis limit
ax2.set_xlim(0,xmax) # Apply limit
ax2.set_xlabel('Separation $r$ (kpc)')
ax2.set_ylabel('Correlation function $\zeta(r)$')
dx=1 # Set width of x-axis bins
ax2.set_ylim(-1,dx*Np*0.5) # Reasonable guess for suitable yaxis scale
xb = np.arange(0,xmax+dx,dx)  # Set x-axis bin edges

corrline, = ax2.plot([],[],drawstyle='steps-post') # Define a command that plots a line for the correlation function 

# for the power spectrum plot 
ax4 = plt.subplot(224) # Create last set of axes as the bottom right panel in a 2x2 grid
ax4.set_xlabel('Wavenumber $k$ (1/kpc)')
ax4.set_ylabel('Power Spectrum $P(r)$')
ax4.set_xlim(1,10) # Set x axis limits - reasonable guess 
# set y axis limits - making reasonable guesses to display useful plots
if (initconfig == "Random Distribution"):
    ax4.set_ylim(-1e3, 1e3)
elif (initconfig == "Lattice (Preset 49 particles)"):
    ax4.set_ylim(-4e3, 4e3)
elif (initconfig == "Two Elliptical Galaxies"):
    ax4.set_ylim(-1e3, 1e3)
ax4.ticklabel_format(axis='y', style='sci', scilimits=(0,0)) # use scientific notation for axes

powerline, = ax4.plot([],[]) # Define a command that plots a line for the power spectrum


In [220]:
# Generate the random distribution for the correlation function ratio 
randomposnozero = np.ravel(np.tril(separation(randompos)))[np.ravel(np.tril(separation(randompos)))>0] # get rid of 0s
hrandom,x = pylab.histogram(randomposnozero,bins=xb) # random distribution to ratio random distribution with 
hrandom = np.array(hrandom, dtype = float) # cast array to a float to be able to return decimals

In [221]:
# Define a function to generate the power spectrum
def powerfuncintegral (k): # set a function to evaluate the power spectrum integral for any k value 
    # Integrate the correlation function to perform a Fourier transform in spherical coords using the trap rule 
    return np.trapz(2*np.pi*x[:-1]**np.sin(np.degrees(k*x[:-1]))/(k)*corrfunc[:], x[:-1])    

In [222]:
# Define the initial function for animation to call to show initial configuration 
def initfunc():
    global position,velocity, corrfunc, powerfunc
    points.set_data(position[0,:], position[1,:]) # Show 2D projection of first 2 position coordinates
    if project_3d:
        points.set_3d_properties(position[2,:])  ## For 3D projection
    
    # Calculate the correlation function 
    posnozero = np.ravel(np.tril(separation(position)))[np.ravel(np.tril(separation(position)))>0] # get rid of 0s
    h,x = pylab.histogram(posnozero,bins=xb) # Make histogram of the lower triangle of the seperation matrix
    h = np.array(h, dtype = float) # cast h to a float to be able to do math on it and return decimals 
    # Use the Peebles-Hause estimator for the correlation function - if RR=0 then set CC/RR=0
    corrfunc = (Nprandom/Np)**2*np.divide(h, hrandom, out = np.zeros_like(h), where=hrandom!=0)-1
    x = np.array(x, dtype = float) # cast to an double to be able to return decimals 
    x[0] = 1e-3; 
    corrline.set_data(x[:-1],corrfunc) # Set the new data for the line in the 2nd panel 
    
    # Calculate the power spectrum
    kvec = np.arange(0.01,11,0.5) # Set up the domain of the waveumber k
    
    # Make the power spectrum function able to return the values to plot in an array 
    vec_pfint = np.vectorize(powerfuncintegral) 
    powerfunc = vec_pfint(kvec) # evaluate power spectrum values with the k vector 
    kvecspline = np.arange(0.1,10,0.05)
#     powerfunc = np.interp(powerfunc, (powerfunc.min(), powerfunc.max()), (-1, +1))
#     powerfunc = (powerfunc - np.min(powerfunc))/(np.max(powerfunc)-np.min(powerfunc))-1
    powerfuncspline = interp1d(kvec, powerfunc, kind='cubic')
    powerline.set_data(kvecspline,powerfuncspline(kvecspline))
    
    return points, corrline, powerline,  # Plot the points and the line


In [223]:
# Define the update function that generates a frame in the animation when called 
def update(i):
    global position,velocity, corrfunc, powerfunc # Get positions and velocities
    if (pmass!=0):
        velocity += acceleration(position)*dt # update velocities using accelerations 
    position += velocity*dt # Increment positions according to their velocites
    position = apply_boundary(position) # apply the boundary conditions 
    
    points.set_data(position[0,:], position[1,:]) # Show 2D projection of first 2 position coordinates
    if project_3d:
        points.set_3d_properties(position[2,:])  ## For 3D projection
    time = str(i)
    ax1.legend([points], [time], loc="upper right", title="Time (Myr)")
    
    
    # Calculate the correlation function 
    posnozero = np.ravel(np.tril(separation(position)))[np.ravel(np.tril(separation(position)))>0] # get rid of 0s
    h,x = pylab.histogram(posnozero,bins=xb) # Make histogram of the lower triangle of the seperation matrix
    h = np.array(h, dtype = float) # cast h to a float to be able to do math on it and return decimals 
    # Use the Peebles-Hause estimator for the correlation function - if RR=0 then set CC/RR=0
    corrfunc = (Nprandom/Np)**2*np.divide(h, hrandom, out = np.zeros_like(h), where=hrandom!=0)-1
    x = np.array(x, dtype = float) # cast to an double to be able to return decimals 
    x[0] = 1e-3; 
    corrline.set_data(x[:-1],corrfunc) # Set the new data for the line in the 2nd panel 
    
    # Calculate the power spectrum
    kvec = np.arange(0.01,11,0.5) # Set up the domain of the waveumber k
    
    # Make the power spectrum function able to return the values to plot in an array 
    vec_pfint = np.vectorize(powerfuncintegral) 
    powerfunc = vec_pfint(kvec) # evaluate power spectrum values with the k vector 
    kvecspline = np.arange(0.1,10,0.05)
    powerfuncspline = interp1d(kvec, powerfunc, kind='cubic')
    powerline.set_data(kvecspline,powerfuncspline(kvecspline))
    
    time_text.set_text('Time: %d Myr' % i*dt);
    return points, corrline, powerline,  # Plot the points and the line


In [224]:
if (initconfig == "Two Elliptical Galaxies"):
        display(Markdown("""# Two Elliptical Galaxies
    Some physics to keep you entertained while this loads 
    """))
elif (initconfig == "Random Distribution"):
    display(Markdown("""# Random Distribution
    Some physics to keep you entertained while this loads 
    """))
elif (initconfig == 'Lattice (Preset 49 particles)'):
    display(Markdown("""# Lattice distribution
    
    Some physics to keep you entertained while this loads 
    """))

# Random Distribution
    Some physics to keep you entertained while this loads 
    

In [225]:
# Generate the animation and display the video 
plt.rcParams['animation.writer'] = 'ffmpeg'
ani = animation.FuncAnimation(fig, update, frames=Nt,interval = frame_duration, init_func = initfunc, blit = False)
HTML(ani.to_html5_video())